# 3 Завдання

## 3.1 Генерування датасету

### Згенеруємо датасет прийому учнів до вищих навчальних закладів.

### Статистика минулих років показує, що в середньому до Університету подають документи 1500 абітурієнтів.

### Для початку імпортуємо модуль NumPy. Кількість згенерованих значення визначенється за допомогою змінної max_applicants, яка дорівнює 1500 + якесь відхилення delta, що визначається випадково у проміжку від -delta / 2 до +delta / 2.

In [110]:
import numpy as np

np.set_printoptions(
    formatter={'float': lambda x: "{0:0.2f}".format(x)})

# delta = 300
max_applicants = 10000
# max_applicants += (np.random.randint(delta) - delta / 2)
# max_applicants = int(max_applicants)

*Визначення загальної кількості абітурієнтів*

### Згенеруємо значення в проміжку від 100 до 200 за допомогою numpy.random.random_integers. Надамо датафрейму назви стовпців.

In [111]:
import pandas as pd
df = pd.DataFrame(
    np.random.randint(100, 200 + 1, (max_applicants, 3)))
df.columns = ['math', 'eng', 'ua']
df

,math,eng,ua
0,197,180,100
1,191,152,178
2,162,148,133
3,197,125,109
4,184,186,131
...,...,...,...
9995,184,142,124
9996,144,142,138
9997,106,102,171
9998,191,182,120


*Список балів абітурієнтів*

### Додамо поле, що визначає рейтинг. Рейтинг абітурієнтів формується за формулою 0,4БМ + 0,3БА + 0,3БУ, де БМ-бал з іспиту з математики, БА-бал з іспиту з англійської мови, БУ-бал з іспиту з української мови.

In [112]:
coefs = np.array([0.4, 0.3, 0.3])
df['score'] = np.apply_along_axis(
    lambda row: np.dot(row, coefs), 1, df.values)
df

,math,eng,ua,score
0,197,180,100,162.8
1,191,152,178,175.4
2,162,148,133,149.1
3,197,125,109,149.0
4,184,186,131,168.7
...,...,...,...,...
9995,184,142,124,153.4
9996,144,142,138,141.6
9997,106,102,171,124.3
9998,191,182,120,167.0


*Додавання поля середнього балу до балів студента*

### Згенеруємо стовпчик про наявність пільг в абітурієнта.

### Очевидно, що студентів з пільгами не може бути багато, то зегенеруємо випадкові значеня у проміжку від 0 до 1. Нехай значення, що менші privilage_quant, визначають, чи абітурієнти мають пільги.

In [113]:
privilage_quant = 0.3
privilige = np.random.random_sample((max_applicants, 1))
privilige = np.apply_along_axis(
    lambda x: x < privilage_quant, 1, privilige)
df['priv'] = privilige
df

,math,eng,ua,score,priv
0,197,180,100,162.8,False
1,191,152,178,175.4,True
2,162,148,133,149.1,False
3,197,125,109,149.0,False
4,184,186,131,168.7,False
...,...,...,...,...,...
9995,184,142,124,153.4,False
9996,144,142,138,141.6,False
9997,106,102,171,124.3,True
9998,191,182,120,167.0,False


*Додавання поля пільг до балів студента*

### Утворимо список значень результатів прийняття абітурієнта до освітнього закладу.

### Мінімальний прохідний бал на вступ 160 для абітурієнтів без пільг.

### З математики для абітурієнтів без пільг мінімальний бал іспиту не може бути менший 140 балів.

### Абітурієнти, які мають пільги, зараховуються при мінімумі 120 балів з усіх іспитів і їх рейтинг не може бути меншим ніж 144 бали.

### Університет може прийняти на навчання 350 абітурієнтів, з них не більше 10% це абітурієнти з пільгами.

In [114]:
mins_no_privilige = np.array([140, 120, 120, 160])
mins_privilige = np.array([120, 120, 120, 144]) 
    
def is_score_valid(row: np.ndarray):
    mins = mins_no_privilige
    if row[4]:
        mins = mins_privilige
    return (row[:4] > mins).all()
df['accepted'] = np.apply_along_axis(
    is_score_valid, 1, df.values)
df

,math,eng,ua,score,priv,accepted
0,197,180,100,162.8,False,False
1,191,152,178,175.4,True,True
2,162,148,133,149.1,False,False
3,197,125,109,149.0,False,False
4,184,186,131,168.7,False,True
...,...,...,...,...,...,...
9995,184,142,124,153.4,False,False
9996,144,142,138,141.6,False,False
9997,106,102,171,124.3,True,False
9998,191,182,120,167.0,False,False


*Визначення прийняття за кількістю балів*

### Оскільки порядок подачі документів абітурієнтами має значення, то додамо стовпець index, де чим він менший, тим раніше абітурієнт подав документи. Зробимо це за допомогою методу reset_index.

### Утворимо відсортований датафрейм за стовпцями в такому порядку used_priv, score. Та визначимо ліміт прийнятих абітурієнтів. Для яких місць не залишиться, тим виставиться False у полі accepted.

In [115]:
df.reset_index(inplace=True)
df.sort_values(['priv', 'score', 'index'], inplace=True,
        ascending=[False, False, True])
df.reset_index(inplace=True, drop=True)
df

,index,math,eng,ua,score,priv,accepted
0,3708,196,195,199,196.6,True,True
1,2334,200,196,188,195.2,True,True
2,7714,199,193,191,194.8,True,True
3,8551,200,195,185,194.0,True,True
4,4919,198,195,186,193.5,True,True
...,...,...,...,...,...,...,...
9995,3443,103,103,109,104.8,False,False
9996,4169,106,105,100,103.9,False,False
9997,9574,105,101,105,103.8,False,False
9998,3038,104,101,104,103.1,False,False


*Індексація та сортування абітурієнтів*

### Визначення студентів з пільгами, що пройшли відбір. Виставлення студентам з пільгами, які не увійши до перших 35-ти людей, параметру accepted = False.

In [116]:
df.loc[(df.priv == True) & (df.index >= 35), 'accepted'] = False
priv = df.loc[(df.priv == True) & (df.accepted == True)]
priv

,index,math,eng,ua,score,priv,accepted
0,3708,196,195,199,196.6,True,True
1,2334,200,196,188,195.2,True,True
2,7714,199,193,191,194.8,True,True
3,8551,200,195,185,194.0,True,True
4,4919,198,195,186,193.5,True,True
5,3084,193,192,195,193.3,True,True
6,8681,192,199,187,192.6,True,True
7,2947,194,187,193,191.6,True,True
8,663,198,200,174,191.4,True,True
9,9786,197,194,181,191.3,True,True


*Перші 35 студентів з пільгами, що пройшли конкурс*

### Визначення результатів студентів без пільг та з пільгами, що не пройшли конкурс.

In [117]:
no_priv = df.loc[(df.priv == False) | (df.accepted == False)]
no_priv.sort_values(['score', 'index'], inplace=True,
        ascending=[False, True])
no_priv.reset_index(inplace=True, drop=True)
no_priv.loc[:350, 'accepted'] = True
no_priv.loc[350:, 'accepted'] = False
no_priv

/tmp/ipykernel_98059/3622713992.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_priv.sort_values(['score', 'index'], inplace=True,


,index,math,eng,ua,score,priv,accepted
0,8467,199,190,200,196.6,False,True
1,2309,200,191,195,195.8,False,True
2,1976,195,197,195,195.6,False,True
3,7871,195,196,195,195.3,False,True
4,1896,200,190,194,195.2,False,True
...,...,...,...,...,...,...,...
9960,3443,103,103,109,104.8,False,False
9961,4169,106,105,100,103.9,False,False
9962,9574,105,101,105,103.8,False,False
9963,3038,104,101,104,103.1,False,False


*Результати вступу*

### Відсортуємо увесь датафрейм за стовпцем index, щоб повернути його до того порядку, з якого починали.

In [118]:
df = pd.concat([priv, no_priv], ignore_index=True)
df.sort_values(['index'], inplace=True, ascending=[True])
df

,index,math,eng,ua,score,priv,accepted
2356,0,197,180,100,162.8,False,False
737,1,191,152,178,175.4,True,False
5143,2,162,148,133,149.1,False,False
5158,3,197,125,109,149.0,False,False
1414,4,184,186,131,168.7,False,False
...,...,...,...,...,...,...,...
4235,9995,184,142,124,153.4,False,False
6799,9996,144,142,138,141.6,False,False
9317,9997,106,102,171,124.3,True,False
1674,9998,191,182,120,167.0,False,False


*Сортування за стовпцем index*

### Очистимо оперативну пам'ять

In [119]:
%reset -f

*Очистка пам'яті*

## 3.2 Навчання моделі

### Навчання моделі будемо проводити в циклі за допомогою перебору параметрів шарів.

### Модель завжди буде компілюватися з optimizer = 'adam', який працює на основі градієнтного спуску, та функцією втрат loss = 'binary_crossentropy', оскільки ці параметри гарно працюють для бінарної класифікації. 

### Оскільки навчання моделі забирає багато часу та ресурсів, то маємо такі обмеження: максимальна кількість шарів знаходиться у layers_numbers; функціями активації можуть бути лише ті, що наведені в activations; кільксть епох знаходиться у epochs_numbers; кількість вхідних нейронів у прихованому шарі знаходиться у neurons_numbers.

### Результати моделей будемо зберігати у список models, що зберігатиме параметри, точності на тренувальних та тестових даних. Запишемо отриманий список у файл.

In [120]:
import numpy as np
import pandas as pd
import copy
layers_numbers = [i for i in range(2, 4)]
activations = ['sigmoid', 'relu']
epochs_numbers = np.array([20, 30, 40]).repeat(3)
neurons_numbers = [50, 100, 200]
def gen_models(layers_numbers, activations,
               epochs_numbers, neurons_numbers):
    models = []
    model = dict(epochs=0, layers=[])
    l = dict(units=0, activation='')
    for layers_num in layers_numbers:
        model['layers'] = [copy.deepcopy(l) for _ in range(layers_num)]
        for epochs in epochs_numbers:
            model['epochs'] = epochs
            for neurons in neurons_numbers:
                for activation in activations:
                    for i, layer in enumerate(model['layers']):
                        layer['units'] = neurons
                        layer['activation'] = activation
                        if i == len(model['layers']) - 1:
                            layer['units'] = 2
                        if i == 0:
                            layer['input_shape'] = (4,)    
                    models.append(format(model))
    return models
    
def write_models(models, output_file):
    with open(output_file, 'w') as file:
        t = '\n'.join(models)
        file.write(t)

*Функція утворення списку можливих комібнацій параметрів*

### Утворимо спискок.

In [121]:
models = gen_models(layers_numbers, activations,
                    epochs_numbers, neurons_numbers)
write_models(models, 'models.txt')
del models

*Утворення списку можливих комібнацій параметрів*

### Розібравши, як створювати датасет, запишемо генерування у функцію та утворимо з неї два набори даних тренувальний та тестовий.

In [122]:
import numpy as np
import pandas as pd
def get_dataset(number: int):
    max_applicants = number
    df = pd.DataFrame(
    np.random.randint(100, 200 + 1, (max_applicants, 3)))
    df.columns = ['math', 'eng', 'ua']
    coefs = np.array([0.4, 0.3, 0.3])
    df['score'] = np.apply_along_axis(
        lambda row: np.dot(row, coefs), 1, df.values)
    privilage_quant = 0.3
    privilige = np.random.random_sample((max_applicants, 1))
    privilige = np.apply_along_axis(
        lambda x: x < privilage_quant, 1, privilige)
    df['priv'] = privilige
    mins_no_privilige = np.array([140, 120, 120, 160])
    mins_privilige = np.array([120, 120, 120, 144]) 
    def is_score_valid(row: np.ndarray):
        mins = mins_no_privilige
        if row[4]:
            mins = mins_privilige
        return (row[:4] > mins).all()
    df['accepted'] = np.apply_along_axis(
        is_score_valid, 1, df.values)
    df.reset_index(inplace=True)
    df.sort_values(['priv', 'score', 'index'], inplace=True,
            ascending=[False, False, True])
    df.reset_index(inplace=True, drop=True)
    df.loc[(df.priv == True) & (df.index >= 35), 'accepted'] = False
    priv = df.loc[(df.priv == True) & (df.accepted == True)]
    no_priv = df.loc[(df.priv == False) | (df.accepted == False)]
    no_priv.sort_values(['score', 'index'], inplace=True,
            ascending=[False, True])
    no_priv.reset_index(inplace=True, drop=True)
    no_priv.loc[:350, 'accepted'] = True
    no_priv.loc[350:, 'accepted'] = False
    df = pd.concat([priv, no_priv], ignore_index=True)
    df.sort_values(['index'], inplace=True, ascending=[True])
    return df.values[:, [1, 2, 3, 4]].astype('float64'), \
        df.values[:, [-1]].astype('float64')

*Функція створення датасету*

### Утворення тренувального та тестового датасетів.


In [123]:
x_train, y_train = get_dataset(2000)
x_test, y_test = get_dataset(1500)
x_train

/tmp/ipykernel_98059/4110377550.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_priv.sort_values(['score', 'index'], inplace=True,
/tmp/ipykernel_98059/4110377550.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_priv.sort_values(['score', 'index'], inplace=True,


array([[156.00, 157.00, 133.00, 149.40],
       [116.00, 198.00, 121.00, 142.10],
       [175.00, 163.00, 193.00, 176.80],
       ...,
       [183.00, 152.00, 112.00, 152.40],
       [135.00, 118.00, 198.00, 148.80],
       [186.00, 109.00, 116.00, 141.90]])

*Навчальні та тестові набори даних*


### Підготуємо мітки за допомогою to_categorical. Тобто 0 перетвориться на [1, 0], а 1 - [0, 1].

In [124]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_train

array([[1.00, 0.00],
       [1.00, 0.00],
       [0.00, 1.00],
       ...,
       [1.00, 0.00],
       [1.00, 0.00],
       [1.00, 0.00]], dtype=float32)

*Підготування міток*

### Нормалізуємо набори даних аогументів. Для початку віднімемо 100 та поділемо на 100.

In [125]:
for arr in [x_train, x_test]:
    arr -= 100
    arr /= 100
x_train

array([[0.56, 0.57, 0.33, 0.49],
       [0.16, 0.98, 0.21, 0.42],
       [0.75, 0.63, 0.93, 0.77],
       ...,
       [0.83, 0.52, 0.12, 0.52],
       [0.35, 0.18, 0.98, 0.49],
       [0.86, 0.09, 0.16, 0.42]])

*Нормалізація даних*

### Запишемо функцію знаходження найкращої моделі.

In [126]:
from ast import literal_eval
from keras import models, layers
best_model = dict(acc=0, model=None)
def find_best_model(input_file, best_model_output_name, best_model):
    with open(input_file, 'r') as file:
        for line in file:
            model = literal_eval(line)
            network = models.Sequential()
            for l in model['layers']:
                network.add(layers.Dense(**l))
            network.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])
            network.fit(x_train, y_train, epochs=model['epochs'])
            test_loss, test_acc = network.evaluate(x_test, y_test)
            if best_model['acc'] < test_acc:
                best_model['model'] = model
                best_model['acc'] = test_acc
                with open(f'{best_model_output_name}.txt', 'w') as bb:
                    bb.write(line)
                network.save(f'{best_model_output_name}.h5')
            del network
            del model

*Функція знаходження найкращої моделі*

### Визначимо найкращу модель. Імпортуємо модуль ast для десеріалізації текстового рядка.

In [127]:
find_best_model('models.txt', 'best_model', best_model)

Epoch 1/20
63/63 [==============================] - 1s 1ms/step - loss: 0.8244 - accuracy: 0.3960
Epoch 2/20
63/63 [==============================] - 0s 2ms/step - loss: 0.5404 - accuracy: 0.8075
Epoch 3/20
63/63 [==============================] - 0s 2ms/step - loss: 0.5103 - accuracy: 0.8075
Epoch 4/20
63/63 [==============================] - 0s 1ms/step - loss: 0.5001 - accuracy: 0.8075
Epoch 5/20
63/63 [==============================] - 0s 2ms/step - loss: 0.4905 - accuracy: 0.8075
Epoch 6/20
63/63 [==============================] - 0s 2ms/step - loss: 0.4803 - accuracy: 0.8075
Epoch 7/20
63/63 [==============================] - 0s 2ms/step - loss: 0.4697 - accuracy: 0.8075
Epoch 8/20
63/63 [==============================] - 0s 2ms/step - loss: 0.4594 - accuracy: 0.8075
Epoch 9/20
63/63 [==============================] - 0s 1ms/step - loss: 0.4483 - accuracy: 0.8075
Epoch 10/20
63/63 [==============================] - 0s 2ms/step - loss: 0.4363 - accuracy: 0.8075
Epoch 11/20
63/63 [

*Визначення найкращої моделі*

### Виведемо найкращу модель.

In [128]:
best_model

{'acc': 0.9879999756813049,
 'model': {'epochs': 30,
  'layers': [{'units': 50, 'activation': 'relu', 'input_shape': (4,)},
   {'units': 50, 'activation': 'relu'},
   {'units': 2, 'activation': 'relu'}]}}

*Найкраща модель*

### Перенесемо прогнози в excel.

In [129]:
from keras.models import load_model
model = load_model('best_model.h5')
pred = model.predict(x_test)
results = (pred[:, 1] > pred[:, 0]).astype('int32')
results = np.hstack([x_test, results.reshape(results.shape[0], 1)])
results = pd.DataFrame(results)
results.columns = ['math', 'eng', 'ua', 'priv', 'accepted']
results.to_excel('result.xlsx')
results

47/47 [==============================] - 0s 1ms/step


,math,eng,ua,priv,accepted
0,0.05,0.28,0.76,0.332,0.0
1,0.77,0.72,0.86,0.782,1.0
2,0.68,0.11,0.82,0.551,0.0
3,0.72,0.11,0.67,0.522,0.0
4,0.44,0.55,0.71,0.554,0.0
...,...,...,...,...,...
1495,0.48,0.63,0.44,0.513,0.0
1496,0.93,0.49,0.13,0.558,0.0
1497,0.54,0.17,0.10,0.297,0.0
1498,0.73,0.71,0.71,0.718,1.0


*Результати моделі*